In [1]:
import os 
import pandas as pd 
import numpy as np 



In [2]:
file_name = 'Maharashtra_AE.csv'
maharashtra_election_df = pd.read_csv(file_name) 

file_name = 'Maharashtra_GA.csv'
maharashtra_ge_election_df = pd.read_csv(file_name) 



/tmp/ipykernel_204051/1564009957.py:2: DtypeWarning: Columns (13,18,19,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  maharashtra_election_df = pd.read_csv(file_name)


In [3]:
maharashtra_ge_election_df.dtypes

Assembly_No                 int64
State_Name                 object
Constituency_No             int64
Poll_No                     int64
Year                        int64
PC_Name                    object
PC_No                       int64
Constituency_Name          object
Constituency_Type          object
CandID                      int64
Candidate                  object
Party                      object
Votes                       int64
Position                    int64
N_Cand                      int64
Valid_Votes                 int64
Vote_Share_Percentage     float64
Deposit_Lost               object
Margin                      int64
Margin_Percentage         float64
ENOP                      float64
Sex                        object
pid                        object
Candidate_Type             object
DelimID                     int64
Party_ID                    int64
Party_Type_TCPD            object
Contested                 float64
Last_Party                 object
Last_Party_ID 

In [9]:
def intermediate_table(df, year):
    constituency = df.iloc[0]['Constituency']
    year = df.iloc[0]['Year']


    # Filter the columns that start with "col_" and include "NOTA"
    col_columns = [col for col in df.columns if col.startswith('col_') or col == 'NOTA']

    # Calculate the sum of each column
    columns_order_by_position = df[col_columns].sum().sort_values(ascending=False).index.tolist()
    # if year in [2014,2019]:
    #     parties_ordered = maharashtra_election_df[(maharashtra_election_df['Constituency_No'] == constituency) & (maharashtra_election_df['Year'] == year)].sort_values('Position', ascending=True)['Party'].tolist()

    if year in [2014]:
        parties_ordered = maharashtra_ge_election_df[(maharashtra_ge_election_df['Constituency_No'] == constituency) & (maharashtra_ge_election_df['Year'] == year)].sort_values('Position', ascending=True)['Party'].tolist()

    # Create a dictionary for renaming columns: match with parties, keep remaining original if excess
    max_len = len(columns_order_by_position)
    rename_dict = {columns_order_by_position[i]: parties_ordered[i] if i < len(parties_ordered) else columns_order_by_position[i] for i in range(max_len)}

    # Rename the columns in the original dataframe
    df.rename(columns=rename_dict, inplace=True)
    if len(parties_ordered) > len(columns_order_by_position): 
        parties_ordered = parties_ordered[:len(columns_order_by_position)]
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    
    # Add INC Status Column
    max_votes_column = df[parties_ordered].idxmax(axis=1)
    df['INC_Status'] = max_votes_column.map(lambda x: 'WON' if x == 'INC' else 'LOSS')

    
    # All columns should be integer columns 
    df['Polling_Station'] = df['Polling_Station'].fillna(df['SN'])

    top_parties = parties_ordered[:3] 

    for party in top_parties:
        df[f'{party} Share%'] = ((df[party] / df['Total']) * 100).round(2)

    # Prepare the columns for the final DataFrame
    final_columns = ['SN', 'Polling_Station', 'Constituency', 'Year'] + [item for party in top_parties for item in (party, f'{party} Share%')] + ['Total','INC_Status']
    final_df = df[final_columns]
    
    return final_df, constituency



In [10]:

import os 


def create_intermediate_tables_and_dump_excel_files(excel_dir, output_dir, log_file_name, year):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(excel_dir):
        if filename.endswith('.xlsx'):
            try: 
                df = pd.read_excel(os.path.join(excel_dir, filename))
                intermediate_df, constituency = intermediate_table(df, year)
                output_file_path = os.path.join(output_dir, f"{constituency}.xlsx")
                intermediate_df.to_excel(output_file_path, index=False)
            except Exception as exc: 
                with open(log_file_name, 'a') as f: 
                    f.write(f"Error processing file: {filename} - {exc}\n")
                continue
                

In [11]:
excel_dir = 'results/cleaned_election_data/MH/GE_2014'
output_dir = 'results/intermediate_tables/MH/GE_2014'
log_file_name = "logs/maharshtra_loksabha_election_intermediate_2014_cleaning_log.txt"
year = 2014

# excel_dir = 'cleaned_election_data/KA/2018'
# output_dir = 'intermediate_tables/KA/2018'
# log_file_name = "logs/karnataka_assembly_election_intermediate_2018_cleaning_log.txt"
# year = 2018

# excel_dir = 'cleaned_election_data/KA/2014'
# output_dir = 'intermediate_tables/KA/2014'
# log_file_name = "logs/karnataka_assembly_election_intermediate_2014_cleaning_log.txt"
# year = 2014

create_intermediate_tables_and_dump_excel_files(excel_dir, output_dir, log_file_name, year)


/tmp/ipykernel_204051/972205591.py:31: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_votes_column = df[parties_ordered].idxmax(axis=1)
/tmp/ipykernel_204051/972205591.py:31: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_votes_column = df[parties_ordered].idxmax(axis=1)
/tmp/ipykernel_204051/972205591.py:31: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_votes_column = df[parties_ordered].idxmax(axis=1)
/tmp/ipykernel_204051/972205591.py:31: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_votes_column = df[parties_ordered].idx

In [26]:
# file_path = 'cleaned_election_data/KA/2014/196.xlsx'
# file_path = 'cleaned_election_data/KA/2018/187.xlsx'
file_path = 'cleaned_election_data/KA/2018/019.xlsx'



df = pd.read_excel(file_path)
print(df.shape)
intermediate_df, constituency = intermediate_table(df, 2014)

print(intermediate_df.shape)


(208, 16)
(208, 12)


/tmp/ipykernel_151147/1371528522.py:31: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_votes_column = df[parties_ordered].idxmax(axis=1)
